In [1]:
from collections import defaultdict
import re
import pprint

def add_key(d, key, value):
    r = dict(d)
    r[key] = value
    return r

def tree():
    return defaultdict(tree)

def dicts(t, node_list): 
    return [add_key(node_list[k], 'node', dicts(t[k],node_list)) for k in t]

def get_level_content(s:str):
    s = re.split('(?<=\!)\!(?!\!)|^\!(?!\!)',s)
    return s[0].count('!'), s[1]

In [ ]:
str = '!(!!/!!)!!health > 0.0147!54: 8'
re.split(r'\(|\)|/', str)

In [67]:
k = []
k.append(1)
print(k)
k.append(2)
print(k)
k.append(3)
print(k)
t = k.pop()
print(k)
k.append(t)
print(k)
print(k[-1])

[1]
[1, 2]
[1, 2, 3]
[1, 2]
[1, 2, 3]
3


In [4]:
def interpret_content(s, node):
    colon = re.search(r':(?= )', s)
    curr_name = ""
    if colon is None:
        t_content = s.split(' ')
        if len(t_content) == 3:
            return {'name':t_content[0], 'op':t_content[1]+t_content[2], 'node':node}
        else:
            print('格式错误: ', s)
            return None
    else:
        t_result = s[colon.span()[0]+1:].strip().split(' ')
        curr_result = int(t_result[0])
        instance_num = re.split(r'\(|\)|/', t_result[1].strip())
        while '' in instance_num:
            instance_num.remove('')
        if len(instance_num) == 2:
            right_num = float(instance_num[0])-float(instance_num[1])
            wrong_num = float(instance_num[1])
        elif len(instance_num) == 1:
            right_num = float(instance_num[0])
            wrong_num = 0
        t_content = s[:colon.span()[0]].strip().split(' ')
        if len(t_content) == 3:
            return {'name':t_content[0], 'op':t_content[1]+t_content[2], "result":curr_result, 'right':right_num, 'wrong':wrong_num, 'node':node}
        else:
            print('格式错误: ', s)
            return None

In [5]:
t = tree()
curr_node = t
node_num = 1
node_list = [{'node':t}]
travel_stack = [0]
last_level = -1
f = open('treeChiMap')
for s in f.readlines():
    curr_level, curr_content = get_level_content(s)
    print(node_num, '\t', curr_level, '\t', travel_stack)
    if(curr_level > last_level):
        curr_node = curr_node[node_num]
    else:
        skip_level = last_level-curr_level+1
        for skip_step in range(skip_level):
            travel_stack.pop()
        curr_node = node_list[travel_stack[-1]]['node'][node_num]
    t_dict = interpret_content(curr_content, curr_node)
    if t_dict is not None:
        node_list.append(t_dict)
    else:
        break
    travel_stack.append(node_num)
    node_num += 1
    last_level = curr_level
res_d = dicts(t, node_list)

1 	 0 	 [0]
2 	 1 	 [0, 1]
3 	 2 	 [0, 1, 2]
4 	 3 	 [0, 1, 2, 3]
5 	 3 	 [0, 1, 2, 3, 4]
6 	 4 	 [0, 1, 2, 3, 5]
7 	 5 	 [0, 1, 2, 3, 5, 6]
8 	 6 	 [0, 1, 2, 3, 5, 6, 7]
9 	 7 	 [0, 1, 2, 3, 5, 6, 7, 8]
10 	 8 	 [0, 1, 2, 3, 5, 6, 7, 8, 9]
11 	 8 	 [0, 1, 2, 3, 5, 6, 7, 8, 9, 10]
12 	 7 	 [0, 1, 2, 3, 5, 6, 7, 8, 9, 11]
13 	 6 	 [0, 1, 2, 3, 5, 6, 7, 8, 12]
14 	 5 	 [0, 1, 2, 3, 5, 6, 7, 13]
15 	 4 	 [0, 1, 2, 3, 5, 6, 14]
16 	 5 	 [0, 1, 2, 3, 5, 15]
17 	 5 	 [0, 1, 2, 3, 5, 15, 16]
18 	 2 	 [0, 1, 2, 3, 5, 15, 17]
19 	 3 	 [0, 1, 2, 18]
20 	 4 	 [0, 1, 2, 18, 19]
21 	 4 	 [0, 1, 2, 18, 19, 20]
22 	 5 	 [0, 1, 2, 18, 19, 21]
23 	 5 	 [0, 1, 2, 18, 19, 21, 22]
24 	 6 	 [0, 1, 2, 18, 19, 21, 23]
25 	 6 	 [0, 1, 2, 18, 19, 21, 23, 24]
26 	 3 	 [0, 1, 2, 18, 19, 21, 23, 25]
27 	 1 	 [0, 1, 2, 18, 26]
28 	 2 	 [0, 1, 27]
29 	 3 	 [0, 1, 27, 28]
30 	 4 	 [0, 1, 27, 28, 29]
31 	 5 	 [0, 1, 27, 28, 29, 30]
32 	 6 	 [0, 1, 27, 28, 29, 30, 31]
33 	 6 	 [0, 1, 27, 28, 29, 30, 31, 32]
34 	 5 	 [

191 	 6 	 [0, 150, 151, 173, 174, 186, 187, 188, 190]
192 	 5 	 [0, 150, 151, 173, 174, 186, 187, 191]
193 	 6 	 [0, 150, 151, 173, 174, 186, 192]
194 	 7 	 [0, 150, 151, 173, 174, 186, 192, 193]
195 	 8 	 [0, 150, 151, 173, 174, 186, 192, 193, 194]
196 	 8 	 [0, 150, 151, 173, 174, 186, 192, 193, 194, 195]
197 	 7 	 [0, 150, 151, 173, 174, 186, 192, 193, 194, 196]
198 	 6 	 [0, 150, 151, 173, 174, 186, 192, 193, 197]
199 	 7 	 [0, 150, 151, 173, 174, 186, 192, 198]
200 	 8 	 [0, 150, 151, 173, 174, 186, 192, 198, 199]
201 	 8 	 [0, 150, 151, 173, 174, 186, 192, 198, 199, 200]
202 	 9 	 [0, 150, 151, 173, 174, 186, 192, 198, 199, 201]
203 	 10 	 [0, 150, 151, 173, 174, 186, 192, 198, 199, 201, 202]
204 	 11 	 [0, 150, 151, 173, 174, 186, 192, 198, 199, 201, 202, 203]
205 	 11 	 [0, 150, 151, 173, 174, 186, 192, 198, 199, 201, 202, 203, 204]
206 	 10 	 [0, 150, 151, 173, 174, 186, 192, 198, 199, 201, 202, 203, 205]
207 	 11 	 [0, 150, 151, 173, 174, 186, 192, 198, 199, 201, 202, 206]
20

In [6]:
import json
res_d
jsObj = json.dumps(res_d)  
  
fileObject = open('jsonFileChi.json', 'w')  
fileObject.write(jsObj)  
fileObject.close()  